# RFM Analysis

This iPython notebook explains how to perform RFM analysis from customer purchase history data.

If you have suggestions or improvements please contribute on https://github.com/joaolcorreia/RFM-analysis

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

Read the sample orders file, containing all past purchases for all customers.

In [2]:
orders = pd.read_csv('sample-orders.csv',sep=',')

In [3]:
orders.head()

,order_date,order_id,customer,grand_total
0,1/1/14 0:00,US-1,Mychal Olson DDS,65
1,1/1/14 0:00,US-2,Ms. Marybeth Breitenberg,68
2,1/1/14 0:00,US-3,Devin Lynch,50
3,1/1/14 0:00,US-4,Allene Nitzsche,76
4,1/1/14 0:00,US-5,Reynaldo Jewess,20


## Create the RFM Table

Since recency is calculated for a given point in time and our sample orders table ranges from Jan 1, 2014 to Mar 31, 2014 we will say NOW is April 1st. 

Set this date to the current day and extract all orders until yesterday.

In [4]:
import datetime as dt
NOW = dt.datetime(2014,4,1)

In [5]:
# Make the date_placed column datetime
orders['order_date'] = pd.to_datetime(orders['order_date'])

Create the RFM Table

In [6]:
rfmTable = orders.groupby('customer').agg({'order_date': lambda x: (NOW - x.max()).days, # Recency
                                        'order_id': lambda x: len(x),      # Frequency
                                        'grand_total': lambda x: x.sum()}) # Monetary Value

rfmTable['order_date'] = rfmTable['order_date'].astype(int)
rfmTable.rename(columns={'order_date': 'recency', 
                         'order_id': 'frequency', 
                         'grand_total': 'monetary_value'}, inplace=True)

## Validating the RFM Table

In [7]:
rfmTable.head()

,frequency,recency,monetary_value
customer,,,
Aaliyah Carroll,1,4,31
Aaliyah Padberg,1,21,76
Aarav Hyatt,1,43,40
Aarav Kassulke,1,70,38
Aarav Kutch,1,72,26


Customer **Aarav Hyatt** has frequency:1, monetary value:$40 and recency:43 days.

In [8]:
aarav = orders[orders['customer']=='Aarav Hyatt']
aarav

,order_date,order_id,customer,grand_total
13676,2014-02-17,US-13677,Aarav Hyatt,40


Inserting the date of Aarav purchase and comparing it to the number of days in the rfmTable we verify our RFM table is correct.

In [9]:
(NOW - dt.datetime(2014,2,17)).days==43

True

## Determining RFM Quartiles

In [10]:
quantiles = rfmTable.quantile(q=[0.25,0.5,0.75])

In [11]:
quantiles

,frequency,recency,monetary_value
0.25,1.0,12.0,44.0
0.50,1.0,38.0,67.0
0.75,2.0,59.0,106.0


Send quantiles to a dictionary, easier to use.

In [12]:
quantiles = quantiles.to_dict()

In [13]:
quantiles

{'frequency': {0.25: 1.0, 0.5: 1.0, 0.75: 2.0},
 'monetary_value': {0.25: 44.0, 0.5: 67.0, 0.75: 106.0},
 'recency': {0.25: 12.0, 0.5: 38.0, 0.75: 59.0}}

## Creating the RFM segmentation table

In [14]:
rfmSegmentation = rfmTable

We create two classes for the RFM segmentation since, being high recency is bad, while high frequency and monetary value is good. 

In [15]:
# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def RClass(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4
    
# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def FMClass(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1


In [16]:
rfmSegmentation['R_Quartile'] = rfmSegmentation['recency'].apply(RClass, args=('recency',quantiles,))
rfmSegmentation['F_Quartile'] = rfmSegmentation['frequency'].apply(FMClass, args=('frequency',quantiles,))
rfmSegmentation['M_Quartile'] = rfmSegmentation['monetary_value'].apply(FMClass, args=('monetary_value',quantiles,))

In [17]:
rfmSegmentation['RFMClass'] = rfmSegmentation.R_Quartile.map(str) \
                            + rfmSegmentation.F_Quartile.map(str) \
                            + rfmSegmentation.M_Quartile.map(str)

In [18]:
rfmSegmentation.head()

,frequency,recency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass
customer,,,,,,,
Aaliyah Carroll,1,4,31,1,4,4,144
Aaliyah Padberg,1,21,76,2,4,2,242
Aarav Hyatt,1,43,40,3,4,4,344
Aarav Kassulke,1,70,38,4,4,4,444
Aarav Kutch,1,72,26,4,4,4,444


In [19]:
# Uncomment any of the following lines to: copy data to clipboard or save it to a CSV file.
# rfmSegmentation.to_clipboard()
# rfmSegmentation.to_csv('rfm-table.csv', sep=',')

Who are the top 5 best customers? by RFM Class (111), high spenders who buy recently and frequently?

In [20]:
rfmSegmentation[rfmSegmentation['RFMClass']=='111'].sort('monetary_value', ascending=False).head(5)

,frequency,recency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass
customer,,,,,,,
Miss Etha Kertzmann,58,4,2869,1,1,1,111
John Mayer,44,1,2270,1,1,1,111
Mrs. Sylva Cormier DDS,24,4,1420,1,1,1,111
Miss Stella Schoen MD,30,3,1398,1,1,1,111
Ms. Tamika Armstrong DDS,28,0,1395,1,1,1,111
